# T1 - Getting started

Installing and getting started with HPVsim is quite simple.

HPVsim is a Python package that can be pip-installed by typing `pip install hpvsim` into a terminal. You can then check that the installation was successful by importing HPVsim with `import hpvsim as hpv`.

The basic design philosophy of HPVsim is: **common tasks should be simple**. For example:

- Defining parameters
- Running a simulation
- Plotting results

This tutorial walks you through how to do these things.

<div class="alert alert-info">
    
Click [here](https://mybinder.org/v2/gh/institutefordiseasemodeling/hpvsim/HEAD?urlpath=lab%2Ftree%2Fdocs%2Ftutorials%2Ftut_intro.ipynb) to open an interactive version of this notebook.
    
</div>

## Getting started

To create, run, and plot a sim with default options is just:

In [ ]:
import hpvsim as hpv

sim = hpv.Sim()
sim.run()
fig = sim.plot()

## Defining parameters and genotypes, and running simulations

Parameters are defined as a dictionary.
Some common parameters to modify are the number of agents in the simulation, the genotypes to simulate, and the start and end dates of the simulation.
We can define those as:

In [ ]:
pars = dict(
    n_agents = 10e3,
    genotypes = [16, 18, 'hr'], # Simulate genotypes 16 and 18, plus all other high-risk HPV genotypes pooled together
    start = 1980,
    end = 2030,
)

Running a simulation is pretty easy. In fact, running a sim with the parameters we defined above is just:

In [ ]:
sim = hpv.Sim(pars)
sim.run()

This will generate a results dictionary `sim.results`.
Results by genotype are named things like `sim.results['infections']` and stored as arrays where each row corresponds to a genotype, while totals across all genotypes have names like `sim.results['infections']` or `sim.results['cancers']`.

Rather than creating a parameter dictionary, any valid parameter can also be passed to the sim directly. For example, exactly equivalent to the above is:

In [ ]:
sim = hpv.Sim(n_agents=10e3, start=1980, end=2030)
sim.run()

You can mix and match too – pass in a parameter dictionary with default options, and then include other parameters as keywords (including overrides; keyword arguments take precedence). For example:

In [ ]:
sim = hpv.Sim(pars, end=2050) # Use parameters defined above, except set the end data to 2050
sim.run()

## Plotting results

As you saw above, plotting the results of a simulation is rather easy too:

In [ ]:
fig = sim.plot()

## Full usage example

Many of the details of this example will be explained in later tutorials, but to give you a taste, here's an example of how you would run two simulations to determine the impact of a custom intervention aimed at protecting the elderly.

In [ ]:
import hpvsim as hpv

# Custom vaccination intervention
def custom_vx(sim):
    if sim.yearvec[sim.t] == 2000:
        target_group = (sim.people.age>9) * (sim.people.age<14)
        sim.people.peak_imm[0, target_group] = 1

pars = dict(
    location = 'tanzania', # Use population characteristics for Japan
    n_agents = 10e3, # Have 50,000 people total in the population
    start = 1980, # Start the simulation in 1980
    n_years = 50, # Run the simulation for 50 years
    burnin = 10, # Discard the first 20 years as burnin period
    verbose = 0, # Do not print any output
)

# Running with multisims -- see Tutorial 3
s1 = hpv.Sim(pars, label='Default')
s2 = hpv.Sim(pars, interventions=custom_vx, label='Custom vaccination')
msim = hpv.MultiSim([s1, s2])
msim.run()
fig = msim.plot(['cancers', 'cins'])